### NOTE FOR LUCA

**Remember to set/remove metadata as:**
{
  "nbsphinx": "hidden"
}

to enable/disable solutions view


# Practical 19

In this practical we will keep working with data structures. In particular, we will see a very important and quite complex data structure called graphs. 

## Slides

The slides of the introduction can be found here: [Intro](docs/Practical19.pdf)

## Graphs

Graphs are mathematical structures made of two key elements: **nodes** (or **vertices**) and **edges**. Nodes are things that we want to represent and edges are relationships among the objects. Mathematically, a graph $G = (N,E)$ where $N$ is a set of nodes and $E = N \times N$ is the set of edges.

Nodes are normally represented as circles, while edges (relationships) are represented by lines/arrows connecting nodes. An example follows:

![](img/pract18/graph_1.png)


Relations represented by edges can be transitive (e.g. sibling_of: if $X$ is sibling of $Y$ then $Y$ is sibling of $X$) and in this case the edges are just lines rather than arrows. In this case the graph is **directed**. In case relationships are not transitive (i.e. $X \rightarrow Y$ does not imply $Y \rightarrow X$) we put an arrow to indicate the direction of the relationship among the nodes and in this case we say the graph is **undirected**.

Some terminology (from the lecture):

![](img/pract18/graphTerms.png)

The **degree** of a node is the number of connection it has with other nodes. In directed graphs the **in-degree** is the number of **incoming** edges, while the **out-degree** is the number of **outgoing** edges. 

![](img/pract18/degree.png)

A **path** in the graph is a sequence of nodes connected by edges. 

![](img/pract18/path.png)

We have seen two different ways of implementing graphs: **adjacency matrices** and **linked lists**. In the remainder, we will work with the **DiGraph** class implemented in the previous practical.


In [1]:
"CODE NOT SHOWN"

#Drawing a graph in pygraphviz
import pygraphviz as pgv

G=pgv.AGraph(directed=True)

#Attributes can be added when adding nodes or edge
G.add_node("Node_1", color='blue')
G.add_node("Node_2", color='blue')
G.add_node("Node_3", color='blue')
G.add_node("Node_4", color='blue')
G.add_node("Node_5", color='blue')
G.add_node("Node_6", color='black')
G.add_edge("Node_1" ,"Node_2", color='blue')
G.add_edge("Node_2" ,"Node_3", color='blue')
G.add_edge("Node_3" ,"Node_4", color='blue')
G.add_edge("Node_4" ,"Node_5", color='blue')
G.add_edge("Node_5" ,"Node_1", color='blue')
G.add_edge("Node_1" ,"Node_6", color='blue')
G.add_edge("Node_2" ,"Node_6", color='blue')
G.add_edge("Node_3" ,"Node_6", color='blue')
G.add_edge("Node_4" ,"Node_6", color='blue')
G.add_edge("Node_5" ,"Node_6", color='blue')
G.add_edge("Node_6" ,"Node_6", color='blue')



# write to a dot file
#G.write('test.dot')

#create a png file
G.layout(prog='fdp') # use dot
G.draw('img/pract18/star.png')

## Visits

Visiting graphs means traversing through its edges and nodes following the connections that make up the graph. Graphs can have cycles and this makes it quite tricky to visit the graph. Differently from what seen in the case of trees, we need to keep a structure of **visited** nodes to avoid getting stuck in loops like the following (or you can pretty much end up in an infinite while loop):

![](img/pract19/circle.png)


Traversing a graph $G = (V,E)$ in mathematical terms can be described as, given a node $r$, visit all the nodes reachable from $r$ going through the nodes exactly once.  

As in the case of Trees, two ways exist to perform a visit of a graph: **depth first search** and **breadth first search**.  

As said above, the base class that we will be extending is [DiGraphLL.py](DiGraphLL/DiGraphLL.py) which is reported below as a reminder:

In [2]:
"""DiGraphLL.py"""

class DiGraphLL:
    def __init__(self):
        """Every node is an element in the dictionary. 
        The key is the node id and the value is a dictionary
        with key second node and value the weight
        """
        self.__nodes = dict()
        
    def insertNode(self, node):
        test = self.__nodes.get(node, None)
        
        if test == None:
            self.__nodes[node] = {}
            #print("Node {} added".format(node))
    
    def insertEdge(self, node1, node2, weight):
        test = self.__nodes.get(node1, None)
        test1 = self.__nodes.get(node2, None)
        if test != None and test1 != None:
            #if both nodes exist othewise don't do anything
            test = self.__nodes[node1].get(node2, None)
            if test != None:
                exStr = "Edge {} --> {} already existing.".format(node1,node2)
                raise Exception(exStr)
            else:    
                #print("Inserted {}-->{} ({})".format(node1,node2,weight))
                self.__nodes[node1][node2] = weight
        
    
    def deleteNode(self, node):
        test = self.__nodes.get(node, None)
        if test != None:
            self.__nodes.pop(node)
        # need to loop through all the nodes!!!
        for n in self.__nodes:
            test = self.__nodes[n].get(node, None)
            if test != None:
                self.__nodes[n].pop(node)
    
    def deleteEdge(self, node1,node2):
        test = self.__nodes.get(node1, None)
        if test != None:
            test = self.__nodes[node1].get(node2, None)
            if test != None:
                self.__nodes[node1].pop(node2)
                
    def __len__(self):
        return len(self.__nodes)
    
    def nodes(self):
        return list(self.__nodes.keys())
    
    def graph(self):
        return self.__nodes
    
    def __str__(self):
        ret = ""
        for n in self.__nodes:
            for edge in self.__nodes[n]:
                
                ret += "{} -- {} --> {}\n".format(str(n),
                                                  str(self.__nodes[n][edge]),
                                                  str(edge))
        return ret
    
    def adjacent(self, node):
        """returns a list of nodes connected to node"""
        ret = []
        test = self.__nodes.get(node, None)
        if test != None:
            for n in self.__nodes:
                if n == node:
                    #all outgoing edges
                    for edge in self.__nodes[node]:
                        ret.append(edge)
                else:
                    #all incoming edges
                    for edge in self.__nodes[n]:
                        if edge == node:
                            ret.append(n)
            
        return ret
    def adjacentEdge(self, node, incoming = True):
        """
        If incoming == False
        we look at the edges of the node
        else we need to loop through all the nodes. 
        An edge is present if there is a 
        corresponding entry in the dictionary.
        If no such nodes exist returns None
        """
        ret = []
        if incoming == False:
            edges = self.__nodes.get(node,None)
            if edges != None:
                for e in edges:
                    w = self.__nodes[node][e]
                    ret.append((node, e, w))
                return ret
        else:
            for n in self.__nodes:
                edge = self.__nodes[n].get(node,None)
                if edge != None:
                    ret.append((n,node, edge))
            return ret
         
    def edges(self):
        """Returns all the edges in a list of triplets"""
        ret = []
        for node in self.__nodes:
            for edge in self.__nodes[node]:
                w = self.__nodes[node][edge]
                ret.append((node,edge, w))
        return ret
 
    def edgeIn(self,node1,node2):
        """Checks if edge node1 --> node2 is present"""
        n1 = self.__nodes.get(node1, None)
        if n1 != None:
            n2 = self.__nodes[node1].get(node2, None)
            if n2 != None:
                return True
            else:
                return False
        else: 
            return False 
        
    
if __name__ == "__main__":
    G = DiGraphLL()
    for i in range(6):
        n = "Node_{}".format(i+1)
        G.insertNode(n)

    for i in range(0,4):
        n = "Node_" + str(i+1)
        six = "Node_6"
        n_plus = "Node_" + str((i+2) % 6)
        G.insertEdge(n, n_plus,0.5)
        G.insertEdge(n, six,1)
    G.insertEdge("Node_5", "Node_1", 0.5)
    G.insertEdge("Node_5", "Node_6", 1)
    G.insertEdge("Node_6", "Node_6", 1)
        
    
    G.insertNode("Node_7")
    G.insertEdge("Node_1", "Node_7", -1)
    G.insertEdge("Node_2", "Node_7", -2)
    G.insertEdge("Node_5", "Node_7", -5)
    G.insertEdge("Node_7", "Node_2", -2)
    G.insertEdge("Node_7", "Node_3", -3)
    

    G.deleteNode("Node_7")
    G.deleteEdge("Node_6", "Node_2")
    #no effect, nodes do not exist!
    G.insertEdge("72", "25",3)
    print(G)
    
    print("\nNodes connected to Node_6:")
    print(G.adjacent("Node_6"))
    print("\nNodes connected to Node_4:")
    print(G.adjacent("Node_4"))
    print("\nNodes connected to Node_3:")
    print(G.adjacent("Node_3"))
    print("Edges outgoing from Node_3:")
    print(G.adjacentEdge("Node_3", incoming = False))
    print("Edges incoming to Node_3:")
    print(G.adjacentEdge("Node_3", incoming = True))
    print("\nEdges incoming to Node_6:")
    print(G.adjacentEdge("Node_6", incoming = True))
    print("\nEdges incoming to Node_743432:")
    print(G.adjacentEdge("Node_743432", incoming = True))
    print("\nAll edges:")

    print(G.edges())
    
    print("\nIs (Node_4,Node_5) there? {}".format( G.edgeIn("Node_4","Node_5")))
    print("Is (Node_4,Node_3) there? {}".format( G.edgeIn("Node_4","Node_3")))
    print("Is (Node_3,Node_4) there? {}".format( G.edgeIn("Node_3","Node_4")))
    print("Is (Node_6,Node_6) there? {}".format( G.edgeIn("Node_6","Node_6")))


Node_2 -- 1 --> Node_6
Node_2 -- 0.5 --> Node_3
Node_5 -- 0.5 --> Node_1
Node_5 -- 1 --> Node_6
Node_1 -- 0.5 --> Node_2
Node_1 -- 1 --> Node_6
Node_3 -- 1 --> Node_6
Node_3 -- 0.5 --> Node_4
Node_6 -- 1 --> Node_6
Node_4 -- 1 --> Node_6
Node_4 -- 0.5 --> Node_5


Nodes connected to Node_6:
['Node_2', 'Node_5', 'Node_1', 'Node_3', 'Node_6', 'Node_4']

Nodes connected to Node_4:
['Node_3', 'Node_6', 'Node_5']

Nodes connected to Node_3:
['Node_2', 'Node_6', 'Node_4']
Edges outgoing from Node_3:
[('Node_3', 'Node_6', 1), ('Node_3', 'Node_4', 0.5)]
Edges incoming to Node_3:
[('Node_2', 'Node_3', 0.5)]

Edges incoming to Node_6:
[('Node_2', 'Node_6', 1), ('Node_5', 'Node_6', 1), ('Node_1', 'Node_6', 1), ('Node_3', 'Node_6', 1), ('Node_6', 'Node_6', 1), ('Node_4', 'Node_6', 1)]

Edges incoming to Node_743432:
[]

All edges:
[('Node_2', 'Node_6', 1), ('Node_2', 'Node_3', 0.5), ('Node_5', 'Node_1', 0.5), ('Node_5', 'Node_6', 1), ('Node_1', 'Node_2', 0.5), ('Node_1', 'Node_6', 1), ('Node_3', '

### Depth First Search (DFS)


### Breadth First Search (BFS)


Breadth First Search visits all the nodes starting from a *root* node level by level. This means that first **all the nodes at distance 1** from the *root* are visited, then **all the nodes at distance 2** and so on.

**Example:** Let's implement BFS for a DiGraph class.

In [3]:
import sys
sys.path.append('DiGraphLL')
import DiGraphLL
from collections import deque

class DiGraph(DiGraphLL.DiGraphLL):
    """
    Every node is an element in the dictionary. 
    The key is the node id and the value is a dictionary
    with key second node and value the weight
    """
    def BFS(self, root):
        if root in self.graph():
            Q = deque()
            Q.append(root)
            #visited is a set of visited nodes
            visited = set()
            visited.add(root)
            while len(Q) > 0:
                curNode = Q.popleft()
                outGoingEdges = self.adjacentEdge(curNode, incoming = False)
                nextNodes = []
                if outGoingEdges != None:
                    #remember that self.adjacentEdge returns:
                    #[('node1','node2', weight1), ...('node1', 'nodeX', weightX)]
                    nextNodes = [x[1] for x in outGoingEdges]
                print("From {}:".format(curNode))
                for nextNode in nextNodes:
                    if nextNode not in visited:
                        Q.append(nextNode)
                        visited.add(nextNode)
                        print("\t --> {}".format(nextNode ))
                        

if __name__ == "__main__":
    G = DiGraph()
    for i in range(1,10):
        G.insertNode("Node_" + str(i))
        
    G.insertEdge("Node_1", "Node_2",1)
    G.insertEdge("Node_2", "Node_1",1)
    G.insertEdge("Node_1", "Node_3",1)
    G.insertEdge("Node_1", "Node_5",1)
    G.insertEdge("Node_2", "Node_3",1)
    G.insertEdge("Node_2", "Node_5",1)
    G.insertEdge("Node_3", "Node_4",1)
    G.insertEdge("Node_3", "Node_6",1)
    G.insertEdge("Node_5", "Node_3",1)
    G.insertEdge("Node_5", "Node_5",1)
    G.insertEdge("Node_6", "Node_4",1)
    G.insertEdge("Node_6", "Node_6",1)
    G.insertEdge("Node_7", "Node_5",1)
    G.insertEdge("Node_5", "Node_8",1)
    G.insertEdge("Node_8", "Node_7",1)
    G.insertEdge("Node_9", "Node_3",1)
    
    G.BFS("Node_1")

                    

From Node_1:
	 --> Node_2
	 --> Node_5
	 --> Node_3
From Node_2:
From Node_5:
	 --> Node_8
From Node_3:
	 --> Node_6
	 --> Node_4
From Node_8:
	 --> Node_7
From Node_6:
From Node_4:
From Node_7:


The graph created is: 

![](img/pract19/testgraph.png)

In [4]:
"CODE NOT SHOWN"

#Drawing a graph in pygraphviz
import pygraphviz as pgv

G=pgv.AGraph(directed=True)

#Attributes can be added when adding nodes or edge
G.add_node("Node_1", color='blue')
G.add_node("Node_2", color='blue')
G.add_node("Node_3", color='blue')
G.add_node("Node_4", color='blue')
G.add_node("Node_5", color='blue')
G.add_node("Node_6", color='blue')
G.add_node("Node_7", color='blue')
G.add_node("Node_8", color='blue')
G.add_node("Node_9", color='blue')
G.add_edge("Node_1" ,"Node_2", color='blue')
G.add_edge("Node_2" ,"Node_1", color='blue')
G.add_edge("Node_1" ,"Node_3", color='blue')
G.add_edge("Node_1" ,"Node_5", color='blue')
G.add_edge("Node_2" ,"Node_3", color='blue')
G.add_edge("Node_2" ,"Node_5", color='blue')
G.add_edge("Node_3" ,"Node_4", color='blue')
G.add_edge("Node_3" ,"Node_6", color='blue')
G.add_edge("Node_5" ,"Node_3", color='blue')
G.add_edge("Node_5" ,"Node_5", color='blue')
G.add_edge("Node_6" ,"Node_4", color='blue')
G.add_edge("Node_6" ,"Node_6", color='blue')
G.add_edge("Node_7" ,"Node_5", color='blue')
G.add_edge("Node_5" ,"Node_8", color='blue')
G.add_edge("Node_8" ,"Node_7", color='blue')
G.add_edge("Node_9" ,"Node_3", color='blue')


# write to a dot file
#G.write('test.dot')

#create a png file
G.layout(prog='dot') # use dot
G.draw('img/pract19/testgraph.png')

## Exercises


1. Find the shortest path between two nodes (if it exists). Hint: 





You can download the code written above to extend it from here: [pract18_ex1.py](file_samples/pract18_ex1.py) 

Test the code with:

```
    G = DiGraphAsAdjacencyMatrix()
    for i in range(6):
        n = "Node_{}".format(i+1)
        G.insertNode(n)

    for i in range(0,4):
        n = "Node_" + str(i+1)
        six = "Node_6"
        n_plus = "Node_" + str((i+2) % 6)
        G.insertEdge(n, n_plus,0.5)
        G.insertEdge(n, six,1)
    G.insertEdge("Node_5", "Node_1", 0.5)
    G.insertEdge("Node_5", "Node_6", 1)
    G.insertEdge("Node_6", "Node_6", 1)
        
    
    G.insertNode("Node_7")
    G.insertEdge("Node_1", "Node_7", -1)
    G.insertEdge("Node_2", "Node_7", -2)
    G.insertEdge("Node_5", "Node_7", -5)
    G.insertEdge("Node_7", "Node_2", -2)
    G.insertEdge("Node_7", "Node_3", -3)
    

    G.deleteNode("Node_7")
    G.deleteEdge("Node_6", "Node_2")
    #no effect, nodes do not exist!
    G.insertEdge("72", "25",3)
    print(G)
    
    print("\nNodes connected to Node_6:")
    print(G.adjacent("Node_6"))
    print("\nNodes connected to Node_4:")
    print(G.adjacent("Node_4"))
    print("\nNodes connected to Node_3:")
    print(G.adjacent("Node_3"))
    print("Edges outgoing from Node_3:")
    print(G.adjacentEdge("Node_3", incoming = False))
    print("Edges incoming to Node_3:")
    print(G.adjacentEdge("Node_3", incoming = True))
    print("\nEdges incoming to Node_6:")
    print(G.adjacentEdge("Node_6", incoming = True))
    print("\nEdges incoming to Node_743432:")
    print(G.adjacentEdge("Node_743432", incoming = True))
    print("\nAll edges:")

    print(G.edges())
    
    print("\nIs (Node_4,Node_5) there? {}".format( G.edgeIn("Node_4","Node_5")))
    print("Is (Node_4,Node_3) there? {}".format( G.edgeIn("Node_4","Node_3")))
    print("Is (Node_3,Node_4) there? {}".format( G.edgeIn("Node_3","Node_4")))
    print("Is (Node_6,Node_6) there? {}".format( G.edgeIn("Node_6","Node_6")))
```

<div class="tggle" onclick="toggleVisibility('ex1');">Show/Hide Solution</div>
<div id="ex1" style="display:none;">

In [5]:
%reset -f

"""
Implementation of a graph as a (weighted) adjacency matrix
store it in: DiGraphAM/DiGraphAM.py
"""

class DiGraphAsAdjacencyMatrix:
    def __init__(self):
        #would be better a set, but I need an index
        self.__nodes = list()
        self.__matrix = list()
        
    def __len__(self):
        """gets the number of nodes"""
        return len(self.__nodes)
        
    def nodes(self):
        return self.__nodes
    
    def matrix(self):
        return self.__matrix
    
    def __str__(self):
        header = "\t".join([n for n in self.__nodes])
        data = ""
        for i in range(0,len(self.__matrix)):
            data += str(self.__nodes[i]) +"\t" + "\t".join([str(x) for x in self.__matrix[i]]) + "\n"

        return "\t"+ header +"\n" + data
    
    def insertNode(self, node):
        #add the node if not there.
        if node not in self.__nodes:
            self.__nodes.append(node)
            #add a row and a column of zeros in the matrix
            if len(self.__matrix) == 0:
                #first node
                self.__matrix = [[0]]
            else:
                N = len(self.__nodes)
                for row in self.__matrix:
                    row.append(0)
                self.__matrix.append([0 for x in range(N)])
    
    def insertEdge(self, node1, node2, weight):
        i = -1
        j = -1
        if node1 in self.__nodes:
            i = self.__nodes.index(node1)
        if node2 in self.__nodes:
            j = self.__nodes.index(node2)
        if i != -1 and j != -1:
            self.__matrix[i][j] = weight
    
    def deleteEdge(self, node1,node2):
        """removing an edge means to set its
        corresponding place in the matrix to 0"""
        i = -1
        j = -1
        if node1 in self.__nodes:
            i = self.__nodes.index(node1)
        if node2 in self.__nodes:
            j = self.__nodes.index(node2)
        if i != -1 and j != -1:
            self.__matrix[i][j] = 0
    
    def deleteNode(self, node):
        """removing a node means removing
        its corresponding row and column in the matrix"""
        i = -1

        if node in self.__nodes:
            i = self.__nodes.index(node)
        #print("Removing {} at index {}".format(node, i))
        if node != -1:
            self.__matrix.pop(i)
            for row in self.__matrix:
                row.pop(i)
            self.__nodes.pop(i)
    
    
    def adjacent(self, node):
        """returns a list of nodes connected to node"""
        ret = []
        if node in self.__nodes:
            i = self.__nodes.index(node)
            #get both incoming and outgoing edges to return nodes
            for j in range(len(self.__nodes)):
                nodeJ = self.__nodes[j]
                if i == j:
                    # outgoing edges!
                    for other in range(len(self.__nodes[i])):
                        nodeO = self.__nodes[other]
                        if self.__matrix[i][other] != 0:
                            ret.append(nodeO)
                else:
                    #incoming edges
                    if self.__matrix[j][i] != 0:
                        ret.append(nodeJ)

        return ret
        
    def adjacentEdge(self, node, incoming = True):
        """
        If incoming == False
        we look at the row of the node
        else at the column. An edge is present if weight
        is different from zero
        """
        ret = []
        i = -1
        if node in self.__nodes:
            i = self.__nodes.index(node)
        if i != -1:
            #if the node is present
            if incoming == False:
                for e in range(len(self.__matrix[i])):
                    edge = self.__nodes[e]
                    w = self.__matrix[i][e]
                    if w != 0:
                        ret.append((node, edge, self.__matrix[i][e]))           
            else:
                for e in range(len(self.__matrix)):
                    edge = self.__nodes[e]
                    w = self.__matrix[e][i]
                    if w != 0:
                        ret.append((edge, node, self.__matrix[e][i]))
            return ret
    
    def edges(self):
        """Returns all the edges in the graph as triplets"""
        ret = []
        for i in range(len(self.__nodes)):
            start = self.__nodes[i]
            for j in range(len(self.__nodes)):
                end = self.__nodes[j]
                w = self.__matrix[i][j]
                if w != 0:
                    ret.append((start, end, w))
        return ret
    
    def edgeIn(self,node1,node2):
        """
        Checks if there exist an edge between node1 and node2
        (i.e. weight != 0)
        """
        if node1 in self.__nodes and node2 in self.__nodes:
            n1 = self.__nodes.index(node1)
            n2 = self.__nodes.index(node2)
            w = n1 = self.__matrix[n1][n2]
            
            if w != 0:
                return True
            else:
                return False
        
        else:
            return False
            
if __name__ == "__main__":
    G = DiGraphAsAdjacencyMatrix()
    for i in range(6):
        n = "Node_{}".format(i+1)
        G.insertNode(n)

    for i in range(0,4):
        n = "Node_" + str(i+1)
        six = "Node_6"
        n_plus = "Node_" + str((i+2) % 6)
        G.insertEdge(n, n_plus,0.5)
        G.insertEdge(n, six,1)
    G.insertEdge("Node_5", "Node_1", 0.5)
    G.insertEdge("Node_5", "Node_6", 1)
    G.insertEdge("Node_6", "Node_6", 1)
        
    
    G.insertNode("Node_7")
    G.insertEdge("Node_1", "Node_7", -1)
    G.insertEdge("Node_2", "Node_7", -2)
    G.insertEdge("Node_5", "Node_7", -5)
    G.insertEdge("Node_7", "Node_2", -2)
    G.insertEdge("Node_7", "Node_3", -3)
    

    G.deleteNode("Node_7")
    G.deleteEdge("Node_6", "Node_2")
    #no effect, nodes do not exist!
    G.insertEdge("72", "25",3)
    print(G)
    
    print("\nNodes connected to Node_6:")
    print(G.adjacent("Node_6"))
    print("\nNodes connected to Node_4:")
    print(G.adjacent("Node_4"))
    print("\nNodes connected to Node_3:")
    print(G.adjacent("Node_3"))
    print("Edges outgoing from Node_3:")
    print(G.adjacentEdge("Node_3", incoming = False))
    print("Edges incoming to Node_3:")
    print(G.adjacentEdge("Node_3", incoming = True))
    print("\nEdges incoming to Node_6:")
    print(G.adjacentEdge("Node_6", incoming = True))
    print("\nEdges incoming to Node_743432:")
    print(G.adjacentEdge("Node_743432", incoming = True))
    print("\nAll edges:")

    print(G.edges())
    
    print("\nIs (Node_4,Node_5) there? {}".format( G.edgeIn("Node_4","Node_5")))
    print("Is (Node_4,Node_3) there? {}".format( G.edgeIn("Node_4","Node_3")))
    print("Is (Node_3,Node_4) there? {}".format( G.edgeIn("Node_3","Node_4")))
    print("Is (Node_6,Node_6) there? {}".format( G.edgeIn("Node_6","Node_6")))

	Node_1	Node_2	Node_3	Node_4	Node_5	Node_6
Node_1	0	0.5	0	0	0	1
Node_2	0	0	0.5	0	0	1
Node_3	0	0	0	0.5	0	1
Node_4	0	0	0	0	0.5	1
Node_5	0.5	0	0	0	0	1
Node_6	0	0	0	0	0	1


Nodes connected to Node_6:
['Node_1', 'Node_2', 'Node_3', 'Node_4', 'Node_5', 'Node_6']

Nodes connected to Node_4:
['Node_3', 'Node_5', 'Node_6']

Nodes connected to Node_3:
['Node_2', 'Node_4', 'Node_6']
Edges outgoing from Node_3:
[('Node_3', 'Node_4', 0.5), ('Node_3', 'Node_6', 1)]
Edges incoming to Node_3:
[('Node_2', 'Node_3', 0.5)]

Edges incoming to Node_6:
[('Node_1', 'Node_6', 1), ('Node_2', 'Node_6', 1), ('Node_3', 'Node_6', 1), ('Node_4', 'Node_6', 1), ('Node_5', 'Node_6', 1), ('Node_6', 'Node_6', 1)]

Edges incoming to Node_743432:
None

All edges:
[('Node_1', 'Node_2', 0.5), ('Node_1', 'Node_6', 1), ('Node_2', 'Node_3', 0.5), ('Node_2', 'Node_6', 1), ('Node_3', 'Node_4', 0.5), ('Node_3', 'Node_6', 1), ('Node_4', 'Node_5', 0.5), ('Node_4', 'Node_6', 1), ('Node_5', 'Node_1', 0.5), ('Node_5', 'Node_6', 1), ('

</div>

2. Consider the Graph class ```DiGraphLL```. Add the following methods:

* ```adjacent(self, node)``` : given a node returns all the nodes connected to it (both incoming and outgoing);

* ```adjacentEdge(self, node, incoming=True)``` : given a node, returns all the nodes close to it (incoming if "incoming=True" or outgoing if "incoming = False") as a list of pairs (node, other, weight);

* ```edges(self)``` : returns all the edges in the graph as pairs (i,j, weight);

* ```edgeIn(self, node1, node2)``` : check if the edge node1 --> node2 is in the graph;

You can download the code written above to extend it from here: [pract18_ex2.py](file_samples/pract18_ex2.py) 

You can test your methods with the code specified in the previous exercise changing ```DiGraphAsAdjacencyMatrix``` with ```DiGraphLL```.

<div class="tggle" onclick="toggleVisibility('ex2');">Show/Hide Solution</div>
<div id="ex2" style="display:none;">

In [6]:
%reset -f 

"""
Implementation of a graph as (weighted) \"linked list\
Write it to: DiGraphLL/DiGraphLL.py
"""

class DiGraphLL:
    def __init__(self):
        """Every node is an element in the dictionary. 
        The key is the node id and the value is a dictionary
        with key second node and value the weight
        """
        self.__nodes = dict()
        
    def insertNode(self, node):
        test = self.__nodes.get(node, None)
        
        if test == None:
            self.__nodes[node] = {}
            #print("Node {} added".format(node))
    
    def insertEdge(self, node1, node2, weight):
        test = self.__nodes.get(node1, None)
        test1 = self.__nodes.get(node2, None)
        if test != None and test1 != None:
            #if both nodes exist othewise don't do anything
            test = self.__nodes[node1].get(node2, None)
            if test != None:
                exStr = "Edge {} --> {} already existing.".format(node1,node2)
                raise Exception(exStr)
            else:    
                #print("Inserted {}-->{} ({})".format(node1,node2,weight))
                self.__nodes[node1][node2] = weight
        
    
    def deleteNode(self, node):
        test = self.__nodes.get(node, None)
        if test != None:
            self.__nodes.pop(node)
        # need to loop through all the nodes!!!
        for n in self.__nodes:
            test = self.__nodes[n].get(node, None)
            if test != None:
                self.__nodes[n].pop(node)
    
    def deleteEdge(self, node1,node2):
        test = self.__nodes.get(node1, None)
        if test != None:
            test = self.__nodes[node1].get(node2, None)
            if test != None:
                self.__nodes[node1].pop(node2)
                
    def __len__(self):
        return len(self.__nodes)
    
    def nodes(self):
        return list(self.__nodes.keys())
    
    def graph(self):
        return self.__nodes
    
    def __str__(self):
        ret = ""
        for n in self.__nodes:
            for edge in self.__nodes[n]:
                
                ret += "{} -- {} --> {}\n".format(str(n),
                                                  str(self.__nodes[n][edge]),
                                                  str(edge))
        return ret
    
    def adjacent(self, node):
        """returns a list of nodes connected to node"""
        ret = []
        test = self.__nodes.get(node, None)
        if test != None:
            for n in self.__nodes:
                if n == node:
                    #all outgoing edges
                    for edge in self.__nodes[node]:
                        ret.append(edge)
                else:
                    #all incoming edges
                    for edge in self.__nodes[n]:
                        if edge == node:
                            ret.append(n)
            
        return ret
    def adjacentEdge(self, node, incoming = True):
        """
        If incoming == False
        we look at the edges of the node
        else we need to loop through all the nodes. 
        An edge is present if there is a 
        corresponding entry in the dictionary.
        If no such nodes exist returns None
        """
        ret = []
        if incoming == False:
            edges = self.__nodes.get(node,None)
            if edges != None:
                for e in edges:
                    w = self.__nodes[node][e]
                    ret.append((node, e, w))
                return ret
        else:
            for n in self.__nodes:
                edge = self.__nodes[n].get(node,None)
                if edge != None:
                    ret.append((n,node, edge))
            return ret
         
    def edges(self):
        """Returns all the edges in a list of triplets"""
        ret = []
        for node in self.__nodes:
            for edge in self.__nodes[node]:
                w = self.__nodes[node][edge]
                ret.append((node,edge, w))
        return ret
 
    def edgeIn(self,node1,node2):
        """Checks if edge node1 --> node2 is present"""
        n1 = self.__nodes.get(node1, None)
        if n1 != None:
            n2 = self.__nodes[node1].get(node2, None)
            if n2 != None:
                return True
            else:
                return False
        else: 
            return False 
    
if __name__ == "__main__":
    G = DiGraphLL()
    for i in range(6):
        n = "Node_{}".format(i+1)
        G.insertNode(n)

    for i in range(0,4):
        n = "Node_" + str(i+1)
        six = "Node_6"
        n_plus = "Node_" + str((i+2) % 6)
        G.insertEdge(n, n_plus,0.5)
        G.insertEdge(n, six,1)
    G.insertEdge("Node_5", "Node_1", 0.5)
    G.insertEdge("Node_5", "Node_6", 1)
    G.insertEdge("Node_6", "Node_6", 1)
        
    
    G.insertNode("Node_7")
    G.insertEdge("Node_1", "Node_7", -1)
    G.insertEdge("Node_2", "Node_7", -2)
    G.insertEdge("Node_5", "Node_7", -5)
    G.insertEdge("Node_7", "Node_2", -2)
    G.insertEdge("Node_7", "Node_3", -3)
    

    G.deleteNode("Node_7")
    G.deleteEdge("Node_6", "Node_2")
    #no effect, nodes do not exist!
    G.insertEdge("72", "25",3)
    print(G)
    
    print("\nNodes connected to Node_6:")
    print(G.adjacent("Node_6"))
    print("\nNodes connected to Node_4:")
    print(G.adjacent("Node_4"))
    print("\nNodes connected to Node_3:")
    print(G.adjacent("Node_3"))
    print("Edges outgoing from Node_3:")
    print(G.adjacentEdge("Node_3", incoming = False))
    print("Edges incoming to Node_3:")
    print(G.adjacentEdge("Node_3", incoming = True))
    print("\nEdges incoming to Node_6:")
    print(G.adjacentEdge("Node_6", incoming = True))
    print("\nEdges incoming to Node_743432:")
    print(G.adjacentEdge("Node_743432", incoming = True))
    print("\nAll edges:")

    print(G.edges())
    
    print("\nIs (Node_4,Node_5) there? {}".format( G.edgeIn("Node_4","Node_5")))
    print("Is (Node_4,Node_3) there? {}".format( G.edgeIn("Node_4","Node_3")))
    print("Is (Node_3,Node_4) there? {}".format( G.edgeIn("Node_3","Node_4")))
    print("Is (Node_6,Node_6) there? {}".format( G.edgeIn("Node_6","Node_6")))

Node_2 -- 1 --> Node_6
Node_2 -- 0.5 --> Node_3
Node_5 -- 0.5 --> Node_1
Node_5 -- 1 --> Node_6
Node_1 -- 0.5 --> Node_2
Node_1 -- 1 --> Node_6
Node_3 -- 1 --> Node_6
Node_3 -- 0.5 --> Node_4
Node_6 -- 1 --> Node_6
Node_4 -- 1 --> Node_6
Node_4 -- 0.5 --> Node_5


Nodes connected to Node_6:
['Node_2', 'Node_5', 'Node_1', 'Node_3', 'Node_6', 'Node_4']

Nodes connected to Node_4:
['Node_3', 'Node_6', 'Node_5']

Nodes connected to Node_3:
['Node_2', 'Node_6', 'Node_4']
Edges outgoing from Node_3:
[('Node_3', 'Node_6', 1), ('Node_3', 'Node_4', 0.5)]
Edges incoming to Node_3:
[('Node_2', 'Node_3', 0.5)]

Edges incoming to Node_6:
[('Node_2', 'Node_6', 1), ('Node_5', 'Node_6', 1), ('Node_1', 'Node_6', 1), ('Node_3', 'Node_6', 1), ('Node_6', 'Node_6', 1), ('Node_4', 'Node_6', 1)]

Edges incoming to Node_743432:
[]

All edges:
[('Node_2', 'Node_6', 1), ('Node_2', 'Node_3', 0.5), ('Node_5', 'Node_1', 0.5), ('Node_5', 'Node_6', 1), ('Node_1', 'Node_2', 0.5), ('Node_1', 'Node_6', 1), ('Node_3', '

</div>

3. Extend the ```DiGraphLL``` class creating a subclass ```DiGraph``` and adding the following methods:

* ```getTopConnected_incoming(self)```:  finds the node with the highest number of in-coming connections;

* ```getTopConnected_outgoing(self)```:  finds the node with the highest number of out-going connections;

* ```hasPath(self, node1,node2)``` to check if there is a path connecting node1 to node2 (if it exists return the path as a list of pair of nodes, otherwise None;

You can test your methods with the following code:

```
G = DiGraphAsAM()
    for i in range(6):
        n = "Node_{}".format(i+1)
        G.insertNode(n)

    for i in range(0,4):
        n = "Node_" + str(i+1)
        six = "Node_6"
        n_plus = "Node_" + str((i+2) % 6)
        G.insertEdge(n, n_plus,0.5)
        G.insertEdge(n, six,1)
    G.insertEdge("Node_5", "Node_1", 0.5)
    G.insertEdge("Node_5", "Node_6", 1)
    G.insertEdge("Node_6", "Node_6", 1)
    print("Top connected (outgoing):")
    print(G.getTopConnected_outgoing())
    print("Top connected (incoming):")
    print(G.getTopConnected_incoming())    
    print("\nAdding edge Node_5 -- 0.5 --> Node_5")
    G.insertEdge("Node_5", "Node_5", 0.5)
    print("Top connected (outgoing):")
    print(G.getTopConnected_outgoing())
    print("\nAre Node_1 and Node_4 connected?")
    print("{}".format(G.hasPath("Node_1","Node_4")))
    print("\nRemoving Node_6")
    G.deleteNode("Node_6")
    print("Top connected (outgoing):")
    print(G.getTopConnected_outgoing())
    print("Top connected (incoming):")
    print(G.getTopConnected_incoming())
    G.insertNode("Node_alone")
    G.insertNode("Node_alone2")
    G.insertEdge("Node_alone", "Node_alone2", 1)
    
    print("\nAre Node_1 and Node_alone2 connected?")
    print(G.hasPath("Node_1", "Node_alone2"))
    print("Are Node_alone2 and Node_alone connected?")
    print(G.hasPath("Node_alone2", "Node_alone"))
```

<div class="tggle" onclick="toggleVisibility('ex3');">Show/Hide Solution</div>
<div id="ex3" style="display:none;">

</div>

In [7]:
%reset -f

import sys
sys.path.append('DiGraphAM')
import DiGraphAM
from collections import deque

class DiGraphAsAM(DiGraphAM.DiGraphAsAdjacencyMatrix):
    
    def getTopConnected_incoming(self):
        topN = ""
        #accumulator to count connections
        conn = [0]*len(self.nodes())
        for node in range(len(self.nodes())):
            for el in range(len(self.matrix()[node])):
                w = self.matrix()[node][el]
                if w != 0:
                    conn[el] +=1
        M = max(conn)
        ind = [x for x in range(len(conn)) if conn[x] == M]
        return [self.nodes()[x] for x in ind]
    
    def getTopConnected_outgoing(self):
        """Returns the node(s)"""
        topN = []
        conn = -1

        for node in range(len(self.nodes())):
            n = len([x for x in self.matrix()[node] if x != 0])
            if n > conn:
                topN = [self.nodes()[node]]
                conn = n
            else:
                if n == conn:
                    topN.append(self.nodes()[node]) 
                
        return topN
    
    def hasPathAux(self, node1,node2):
        if node1 not in self.nodes() or node2 not in self.nodes():
            return False
        else:
            Q = deque()
            Q.append(node1)
            visited = set()
            i2 = self.nodes().index(node2)
            while len(Q) > 0:
                curN = Q.popleft()
                i1 = self.nodes().index(curN)
                #do not travel on already visited nodes
                if curN not in visited:
                    visited.add(curN)
                    #get all outgoing nodes of Q
                    for edge in range(len(self.matrix()[i1])):
                        w = self.matrix()[i1][edge]
                        if w != 0:
                            if edge == i2:
                                return True
                            else:
                                Q.append(self.nodes()[edge])
            
            return False
        
    def hasPath(self, node1, node2):
        #checks both paths and returns True or false
        res = self.hasPathAux(node1,node2)
        if res:
            return True
        else:
            return self.hasPathAux(node2,node1)
    

if __name__ == "__main__":    
    G = DiGraphAsAM()
    for i in range(6):
        n = "Node_{}".format(i+1)
        G.insertNode(n)

    for i in range(0,4):
        n = "Node_" + str(i+1)
        six = "Node_6"
        n_plus = "Node_" + str((i+2) % 6)
        G.insertEdge(n, n_plus,0.5)
        G.insertEdge(n, six,1)
    G.insertEdge("Node_5", "Node_1", 0.5)
    G.insertEdge("Node_5", "Node_6", 1)
    G.insertEdge("Node_6", "Node_6", 1)
    print("Top connected (outgoing):")
    print(G.getTopConnected_outgoing())
    print("Top connected (incoming):")
    print(G.getTopConnected_incoming())    
    print("\nAdding edge Node_5 -- 0.5 --> Node_5")
    G.insertEdge("Node_5", "Node_5", 0.5)
    print("Top connected (outgoing):")
    print(G.getTopConnected_outgoing())
    print("\nAre Node_1 and Node_4 connected?")
    print("{}".format(G.hasPath("Node_1","Node_4")))
    print("\nRemoving Node_6")
    G.deleteNode("Node_6")
    print("Top connected (outgoing):")
    print(G.getTopConnected_outgoing())
    print("Top connected (incoming):")
    print(G.getTopConnected_incoming())
    G.insertNode("Node_alone")
    G.insertNode("Node_alone2")
    G.insertEdge("Node_alone", "Node_alone2", 1)
    
    print("\nAre Node_1 and Node_alone2 connected?")
    print(G.hasPath("Node_1", "Node_alone2"))
    print("Are Node_alone2 and Node_alone connected?")
    print(G.hasPath("Node_alone2", "Node_alone"))
    

Top connected (outgoing):
['Node_1', 'Node_2', 'Node_3', 'Node_4', 'Node_5']
Top connected (incoming):
['Node_6']

Adding edge Node_5 -- 0.5 --> Node_5
Top connected (outgoing):
['Node_5']

Are Node_1 and Node_4 connected?
True

Removing Node_6
Top connected (outgoing):
['Node_5']
Top connected (incoming):
['Node_5']

Are Node_1 and Node_alone2 connected?
False
Are Node_alone2 and Node_alone connected?
True


4. Extend the ```DiGraphLL``` class creating a subclass ```DiGraph``` by adding the following methods:

* ```getTopConnected_incoming(self)```:  finds the node with the highest number of in-coming connections;

* ```getTopConnected_outgoing(self)```:  finds the node with the highest number of out-going connections;

* ```hasPath(self, node1,node2)``` to check if there is a path connecting node1 to node2 (if it exists return the path as a list of pair of nodes, otherwise None;

Test your class with the following code:

```
G = DiGraph()
    for i in range(6):
        n = "Node_{}".format(i+1)
        G.insertNode(n)

    for i in range(0,4):
        n = "Node_" + str(i+1)
        six = "Node_6"
        n_plus = "Node_" + str((i+2) % 6)
        G.insertEdge(n, n_plus,0.5)
        G.insertEdge(n, six,1)
    G.insertEdge("Node_5", "Node_1", 0.5)
    G.insertEdge("Node_5", "Node_6", 1)
    G.insertEdge("Node_6", "Node_6", 1)
    print("Top connected (outgoing):")
    print(G.getTopConnected_outgoing())
    print("Top connected (incoming):")
    print(G.getTopConnected_incoming())    
    print("\nAdding edge Node_5 -- 0.5 --> Node_5")
    G.insertEdge("Node_5", "Node_5", 0.5)
    print("Top connected (outgoing):")
    print(G.getTopConnected_outgoing())
    print("\nAre Node_1 and Node_4 connected?")
    print("{}".format(G.hasPath("Node_1","Node_4")))
    print("\nRemoving Node_6")
    G.deleteNode("Node_6")
    print("Top connected (outgoing):")
    print(G.getTopConnected_outgoing())
    print("Top connected (incoming):")
    print(G.getTopConnected_incoming())
    G.insertNode("Node_alone")
    G.insertNode("Node_alone2")
    G.insertEdge("Node_alone", "Node_alone2", 1)
    
    print("\nAre Node_1 and Node_alone2 connected?")
    print(G.hasPath("Node_1", "Node_alone2"))
    print("Are Node_alone2 and Node_alone connected?")
    print(G.hasPath("Node_alone2", "Node_alone"))
```

<div class="tggle" onclick="toggleVisibility('ex4');">Show/Hide Solution</div>
<div id="ex4" style="display:none;">

In [8]:
%reset -f 

import sys
sys.path.append('DiGraphLL')
import DiGraphLL
from collections import deque

class DiGraph(DiGraphLL.DiGraphLL):
    """Every node is an element in the dictionary. 
        The key is the node id and the value is a dictionary
        with key second node and value the weight
        """
    
    def getTopConnected_incoming(self):
        
        topN = ""
        #accumulator to count connections
        conn = [0]*len(self.nodes())
        for node in self.nodes():
            for el in self.graph()[node]:
                elInd = self.nodes().index(el)
                conn[elInd] +=1
        M = max(conn)
        ind = [x for x in range(len(conn)) if conn[x] == M]
        return [self.nodes()[x] for x in ind]
    
    def getTopConnected_outgoing(self):
        """Returns the node(s)"""
        topN = []
        conn = -1

        for node in self.nodes():
            n = len(self.graph()[node])
            if n > conn:
                topN = [node]
                conn = n
            else:
                if n == conn:
                    topN.append(node) 
                
        return topN
    
    def hasPathAux(self, node1,node2):
        if node1 not in self.nodes() or node2 not in self.nodes():
            return False
        else:
            Q = deque()
            Q.append(node1)
            visited = set()
            while len(Q) > 0:
                curN = Q.popleft()
                #do not travel on already visited nodes
                if curN not in visited:
                    visited.add(curN)
                    #get all outgoing nodes of Q
                    for edge in self.graph()[curN]:
                        if edge == node2:
                            return True
                        else:
                            Q.append(edge)
            
            return False
        
    def hasPath(self, node1, node2):
        #checks both paths and returns True or false
        res = self.hasPathAux(node1,node2)
        if res:
            return True
        else:
            return self.hasPathAux(node2,node1)
    

if __name__ == "__main__":    
    G = DiGraph()
    for i in range(6):
        n = "Node_{}".format(i+1)
        G.insertNode(n)

    for i in range(0,4):
        n = "Node_" + str(i+1)
        six = "Node_6"
        n_plus = "Node_" + str((i+2) % 6)
        G.insertEdge(n, n_plus,0.5)
        G.insertEdge(n, six,1)
    G.insertEdge("Node_5", "Node_1", 0.5)
    G.insertEdge("Node_5", "Node_6", 1)
    G.insertEdge("Node_6", "Node_6", 1)
    print("Top connected (outgoing):")
    print(G.getTopConnected_outgoing())
    print("Top connected (incoming):")
    print(G.getTopConnected_incoming())    
    print("\nAdding edge Node_5 -- 0.5 --> Node_5")
    G.insertEdge("Node_5", "Node_5", 0.5)
    print("Top connected (outgoing):")
    print(G.getTopConnected_outgoing())
    print("\nAre Node_1 and Node_4 connected?")
    print("{}".format(G.hasPath("Node_1","Node_4")))
    print("\nRemoving Node_6")
    G.deleteNode("Node_6")
    print("Top connected (outgoing):")
    print(G.getTopConnected_outgoing())
    print("Top connected (incoming):")
    print(G.getTopConnected_incoming())
    print("\nAdding Node_alone and Node_alone2")
    G.insertNode("Node_alone")
    G.insertNode("Node_alone2")
    print("Adding Node_alone --> Node_alone2")
    G.insertEdge("Node_alone", "Node_alone2", 1)
    
    print("\nAre Node_1 and Node_alone2 connected?")
    print(G.hasPath("Node_1", "Node_alone2"))
    print("Are Node_alone2 and Node_alone connected?")
    print(G.hasPath("Node_alone2", "Node_alone"))
    

Top connected (outgoing):
['Node_2', 'Node_5', 'Node_1', 'Node_3', 'Node_4']
Top connected (incoming):
['Node_6']

Adding edge Node_5 -- 0.5 --> Node_5
Top connected (outgoing):
['Node_5']

Are Node_1 and Node_4 connected?
True

Removing Node_6
Top connected (outgoing):
['Node_5']
Top connected (incoming):
['Node_5']

Adding Node_alone and Node_alone2
Adding Node_alone --> Node_alone2

Are Node_1 and Node_alone2 connected?
False
Are Node_alone2 and Node_alone connected?
True


</div>